In [14]:
import requests
import time
import re
import pandas as pd
from tqdm import tqdm
import pymysql
from multiprocessing.dummy import Pool
key = "ckey_2545e4a689bb4d4c87d64889755"
# 0xAD0c640dA6A14e1Bb9A48d95609f83982767A047   zk
# 0x55d398326f99059fF775485246999027B3197955   usdt

class GetERC20:
    def __init__(self,addr):
        #合约地址
        self.addr = addr
        self.contract_address = contract_address
        
    def request(self):
        """
        请求网页
        """
        # global error_url
        global ans
        page_number = 0
        page_size = 1000
        while True:
            params = {
                "page-number":f"{page_number}",
                "page-size":f"{page_size}",
                "key":"ckey_2545e4a689bb4d4c87d64889755",  #密钥
                'contract-address':self.contract_address  #代币地址
                }
            url = f"https://api.covalenthq.com/v1/56/address/{self.addr}/transfers_v2/?quote-currency=USD"
            try:
                r = requests.get(url,params=params)
            except Exception as e:
                # print(f"捕获异常{e}")
                # print("捕获异常，等待 {} 秒后继续尝试请求！".format(3))
                # time.sleep(3)
                print(r)
                continue
            
            if (r.status_code == 200) and (len(r.json()['data']['items'])>0):
                for i in tqdm(r.json()['data']['items']):
                    if i['successful'] == True:
                        tx_hash = i['tx_hash']
                        from_address = i['transfers'][0]['from_address']
                        to_address = i['transfers'][0]['to_address']
                        value_ = i['transfers'][0]['delta']
                        contract_ticker_symbol = i['transfers'][0]['contract_ticker_symbol']
                        transfer_type = i['transfers'][0]['transfer_type']
                        ans['tx_hash'].append(tx_hash)
                        ans['from'].append(from_address)
                        ans['to'].append(to_address)
                        ans['value'].append(value_)
                        ans['symbol'].append(contract_ticker_symbol)
                        ans['transfer_type'].append(transfer_type)

                page_number+=1
                # age = len(ans['tx_hash'])
                # if age//2500>0:
                #     print(f"已爬取{age}条")
            elif (r.status_code == 200) and (len(r.json()['data']['items'])==0):
                print('爬取完毕')
                break
            elif r.status_code != 200: 
                time.sleep(5)
                continue
                

    def save(self,ans):
        return ans.to_excel(f'./result/{self.addr}.xlsx',index=False)
        
        


# addr = "0xbdd030fa56eb397c528e11499b5f6f5a10ed1d82"  #池子地址
# contract_address="0x55d398326f99059fF775485246999027B3197955"    #交易代币地址usdt/BNB

addr = "0xB8ba2032af48a7cbdd2A5F5687A84b7D26Be5ff1"  #池子地址
contract_address="0x55d398326f99059fF775485246999027B3197955"    #交易代币地址usdt/BNB
ans = {'transfer_type':[],'from':[],'to':[],'value':[],'symbol':[],'tx_hash':[]}

ER = GetERC20(addr)
ER.request()

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 5043.66it/s]


爬取完毕


In [15]:
df = pd.DataFrame(ans)
df['len'] = df['value'].map(lambda i:len(i))
df['len'].min()
df['value'] = df['value'].map(lambda i:int(i[:-10])/100000000)

In [16]:
df.groupby(['from','to'],as_index=False)['value'].sum().sort_values('value',ascending=False).reset_index(drop=True).head(10)

,from,to,value
0,0x9d7008709084feca4910fc0f82855fdba584babe,0xb8ba2032af48a7cbdd2a5f5687a84b7d26be5ff1,603600.584054
1,0xb8ba2032af48a7cbdd2a5f5687a84b7d26be5ff1,0xe3164706acc4623b8289f7f7c0445ce0e08af3bc,40000.000000
2,0xb8ba2032af48a7cbdd2a5f5687a84b7d26be5ff1,0xdcb5c319259d47cdbc003a638b2aa5c3c9dd8c53,3600.000000


In [5]:
df.groupby(['to'],as_index=False)['value'].sum().sort_values('value',ascending=False).reset_index(drop=True).head(10)

,to,value
0,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,6.173836e+06
1,0x16b9a82891338f9ba80e2d6970fdda79d1eb0dae,7.636597e+05
2,0xbbb5a3829c29c8b49b82a8941447df645fadc8ac,3.897776e+05
3,0x1ae1d599c556c087eb6348c12d9cd114868f6e34,2.527250e+05
4,0x55285b8b32cdfb52fde66fc81f8bcd83b723148f,1.104285e+05
5,0xc4e202960fe384eef9188be81cb1730d151083a5,9.999550e+04
6,0x923008dd433fc653fc5d856c74ea1a877fc5aee3,9.301700e+04
7,0xc13258e384ec9312c8f42ea655a5dbe3cec4c185,9.230830e+04
8,0x5efd59a3c5af0a7efd5767e13e6592ac172f0306,5.913610e+04
9,0x7570a317daad1552e323c3a9c8f3937f8e0beb3c,5.873660e+04


In [18]:
df = pd.read_csv('0x5799Cc9bC780e529d8B4e64b9E8C5505923a8005.csv')
df['count'] = 1

In [27]:
df.head(1)

,transfer_type,from,to,value,symbol,tx_hash,len,count
0,IN,0xfa52f15c4e0b282b47d51b8eef6d15d830e7b5e9,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,38.0,USDT,0x3d5af792982b9d91a91e251ce1a532907b88a6e870b7...,20,1


In [23]:
df.groupby(['from','to'],as_index=False)['value'].sum().sort_values('value',ascending=False).reset_index(drop=True).head(20)['from'].unique()

array(['0x5799cc9bc780e529d8b4e64b9e8c5505923a8005',
       '0xa180fe01b906a1be37be6c534a3300785b20d947',
       '0xbbb5a3829c29c8b49b82a8941447df645fadc8ac',
       '0xeb2d2f1b8c558a40207669291fda468e50c8a0bb',
       '0xe2fc31f816a9b94326492132018c3aecc4a93ae1',
       '0x8894e0a0c962cb723c1976a4421c95949be2d4e3',
       '0x4982085c9e2f89f2ecb8131eca71afad896e89cb',
       '0x8dc3eb393f110ebab45d47a9a32c4c0c0b105bb2',
       '0xfea47143d1862a8c6419d5043a33cd1e3cbe9265',
       '0xe7b5fff879430e6e831001fda7cd06ccd17ab7fa',
       '0xf0d5d2b91641dd3541085690ba03ad7d955d4cf9',
       '0xe3164706acc4623b8289f7f7c0445ce0e08af3bc',
       '0x5efd59a3c5af0a7efd5767e13e6592ac172f0306',
       '0xd4c334075a3f648d4c3cbed63c38276c70c92e12'], dtype=object)

In [40]:
df.groupby(['from','to'],as_index=False)[['value','count']].sum().sort_values('value',ascending=False).reset_index(drop=True).head(20)

,from,to,value,count
0,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,0x16b9a82891338f9ba80e2d6970fdda79d1eb0dae,763659.658035,1469
1,0xa180fe01b906a1be37be6c534a3300785b20d947,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,480028.256292,158
2,0xbbb5a3829c29c8b49b82a8941447df645fadc8ac,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,421225.170000,306
3,0xeb2d2f1b8c558a40207669291fda468e50c8a0bb,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,393304.601933,132
4,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,0xbbb5a3829c29c8b49b82a8941447df645fadc8ac,389777.600000,308
5,0xe2fc31f816a9b94326492132018c3aecc4a93ae1,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,334459.403385,114
6,0x8894e0a0c962cb723c1976a4421c95949be2d4e3,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,311200.680022,109
7,0x4982085c9e2f89f2ecb8131eca71afad896e89cb,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,298491.041844,111
8,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,0x1ae1d599c556c087eb6348c12d9cd114868f6e34,252725.000000,84
9,0x8dc3eb393f110ebab45d47a9a32c4c0c0b105bb2,0x5799cc9bc780e529d8b4e64b9e8c5505923a8005,231776.870566,9


In [29]:
df[df['transfer_type']=="IN"].groupby(['from'],as_index=False)[['value','count']].sum().sort_values('count',ascending=False)

,from,value,count
865,0xbbb5a3829c29c8b49b82a8941447df645fadc8ac,421225.170000,306
746,0xa180fe01b906a1be37be6c534a3300785b20d947,480028.256292,158
1094,0xeb2d2f1b8c558a40207669291fda468e50c8a0bb,393304.601933,132
1041,0xe2fc31f816a9b94326492132018c3aecc4a93ae1,334459.403385,114
339,0x4982085c9e2f89f2ecb8131eca71afad896e89cb,298491.041844,111
...,...,...,...
585,0x7f743ab17de0e4275b511e4d94fb2c1a45acc9de,700.000000,1
586,0x7fa0fbd495ed2792d6ad5aa6d979903b93d57b5c,57.975166,1
587,0x7fc83f2fa264fd6395d1f3f86f06bed17fe70f97,1000.000000,1
588,0x7fdad624a700787a2be90767156a96a46aec0993,990.000000,1


In [31]:
df[df['transfer_type']=="OUT"].groupby(['to'],as_index=False)[['value','count']].sum().sort_values('count',ascending=False)

,to,value,count
335,0x16b9a82891338f9ba80e2d6970fdda79d1eb0dae,763659.658035,1469
2825,0xbbb5a3829c29c8b49b82a8941447df645fadc8ac,389777.600000,308
399,0x1ae1d599c556c087eb6348c12d9cd114868f6e34,252725.000000,84
2544,0xaae1b6ef53bb45684bb753317223c32cee5045da,7490.700000,79
1508,0x645beaa636aa50424b496967b27de0271e3022a4,18017.000000,79
...,...,...,...
2157,0x92604f91eacdc23c5827d4d09b7773a93d13707a,76.000000,1
874,0x3838b8480f86ba5db356a7d695ac777cef1e1417,312.500000,1
871,0x380221523bcd438542136cca512ca7a920d26d4c,100.000000,1
870,0x37fc55cbfd5948b9b3365a08973678c5f4db0ce1,300.000000,1
